# Homework 6

### Dataset

In this homework, we will use the California Housing Prices from [Kaggle](https://www.kaggle.com/datasets/camnugent/california-housing-prices).

In [1]:
# !wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv

--2023-10-20 09:17:21--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8002::154, 2606:50c0:8001::154, 2606:50c0:8000::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8002::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1423529 (1.4M) [text/plain]
Saving to: ‘housing.csv’

housing.csv         100%[===================>]   1.36M  3.54MB/s    in 0.4s    

2023-10-20 09:17:22 (3.54 MB/s) - ‘housing.csv’ saved [1423529/1423529]



In [56]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

### Preparing the dataset 

For this homework, we only want to use a subset of data. This is the same subset we used in homework #2.
But in contrast to homework #2, we are going to use all columns of the dataset.

First, keep only the records where `ocean_proximity` is either `'<1H OCEAN'` or `'INLAND'`

Preparation:

* Fill missing values with zeros.
* Apply the log transform to `median_house_value`.
* Do train/validation/test split with 60%/20%/20% distribution. 
* Use the `train_test_split` function and set the `random_state` parameter to 1.
* Use `DictVectorizer(sparse=True)` to turn the dataframes into matrices.

In [5]:
df = pd.read_csv('housing.csv')
df.columns = df.columns.str.replace(' ', '_').str.lower()
df = df.loc[df['ocean_proximity'].isin(['<1H OCEAN', 'INLAND'])]
df = df.fillna(0)
df.median_house_value = np.log1p(df.median_house_value.values)
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
701,-121.97,37.64,32.0,1283.0,194.0,485.0,171.0,6.0574,12.973866,<1H OCEAN
830,-121.99,37.61,9.0,3666.0,711.0,2341.0,703.0,4.6458,12.287657,<1H OCEAN
859,-121.97,37.57,21.0,4342.0,783.0,2172.0,789.0,4.6146,12.419574,<1H OCEAN
860,-121.96,37.58,15.0,3575.0,597.0,1777.0,559.0,5.7192,12.554971,<1H OCEAN
861,-121.98,37.58,20.0,4126.0,1031.0,2079.0,975.0,3.6832,12.287196,<1H OCEAN


In [6]:
def train_val_test_split(df, target, train_size, val_size, test_size, random_state):
    
    df_full_train, df_test = train_test_split(df, test_size=test_size, random_state=random_state)
    val_portion = val_size / (train_size + val_size)
    df_train, df_val = train_test_split(df_full_train, test_size=val_portion, random_state=random_state)

    df_train = df_train.reset_index(drop=True)
    df_val = df_val.reset_index(drop=True)
    df_test = df_test.reset_index(drop=True)

    y_train = df_train[target].values
    y_val = df_val[target].values
    y_test = df_test[target].values

    del df_train[target]
    del df_val[target]
    del df_test[target]

    return df_full_train, df_train, df_test, df_val, y_train, y_val, y_test

In [7]:
df_full_train, df_train, df_test, df_val, y_train, y_val, y_test = \
    train_val_test_split(df=df, target='median_house_value', train_size=0.6, val_size=0.2, test_size=0.2, random_state=1)


## Question 1

Let's train a decision tree regressor to predict the `median_house_value` variable. 

* Train a model with `max_depth=1`.


Which feature is used for splitting the data?


In [18]:
def train(df_train, y_train, max_depth):
    dicts = df_train.to_dict(orient='records')

    dv = DictVectorizer(sparse=True)
    X_train = dv.fit_transform(dicts)

    model = DecisionTreeRegressor(max_depth=max_depth)
    model.fit(X_train, y_train)
    
    return dv, model

In [26]:
dv, model = train(df_train=df_train, y_train=y_train, max_depth=1)
importances = list(zip(dv.feature_names_, model.feature_importances_))
df_importance = pd.DataFrame(importances, columns=['feature', 'gain'])
df_importance = df_importance.sort_values(by='gain', ascending=False)
df_importance

,feature,gain
5,ocean_proximity=<1H OCEAN,1.0
0,households,0.0
1,housing_median_age,0.0
2,latitude,0.0
3,longitude,0.0
4,median_income,0.0
6,ocean_proximity=INLAND,0.0
7,population,0.0
8,total_bedrooms,0.0
9,total_rooms,0.0


## Question 2

Train a random forest model with these parameters:

* `n_estimators=10`
* `random_state=1`
* `n_jobs=-1` (optional - to make training faster)

What's the RMSE of this model on validation?

In [42]:
def train_rf(df_train, y_train, n_estimators):
    dicts = df_train.to_dict(orient='records')

    dv = DictVectorizer(sparse=True)
    X_train = dv.fit_transform(dicts)

    model = RandomForestRegressor(n_estimators=n_estimators, random_state=1, n_jobs=-1)
    model.fit(X_train, y_train)
    
    return dv, model

def predict(df, dv, model):
    dicts = df.to_dict(orient='records')

    X = dv.transform(dicts)
    y_pred = model.predict(X)

    return y_pred

def rmse(y, y_pred):
    error = y_pred - y
    mse = (error ** 2).mean()
    return np.sqrt(mse)

In [43]:
dv, rf_model = train_rf(df_train=df_train, y_train=y_train, n_estimators=10)
y_pred = predict(df=df_val, dv=dv, model=rf_model)
round(rmse(y_val, y_pred),3)

0.245

## Question 3

Now let's experiment with the `n_estimators` parameter

* Try different values of this parameter from 10 to 200 with step 10.
* Set `random_state` to `1`.
* Evaluate the model on the validation dataset.


After which value of `n_estimators` does RMSE stop improving?
Consider 3 decimal places for retrieving the answer.


In [45]:
all_rmse = {}

for i in range(10, 201, 10):
    dv, rf_model = train_rf(df_train=df_train, y_train=y_train, n_estimators=i)
    y_pred = predict(df=df_val, dv=dv, model=rf_model)
    rmse_val = round(rmse(y_val, y_pred),3)
    print('%s -> %.3f' % (i, rmse_val))
    all_rmse[i] = rmse_val

10 -> 0.245
20 -> 0.239
30 -> 0.237
40 -> 0.235
50 -> 0.235
60 -> 0.234
70 -> 0.234
80 -> 0.234
90 -> 0.234
100 -> 0.234
110 -> 0.234
120 -> 0.234
130 -> 0.234
140 -> 0.233
150 -> 0.233
160 -> 0.233
170 -> 0.233
180 -> 0.233
190 -> 0.234
200 -> 0.234



## Question 4

Let's select the best `max_depth`:

* Try different values of `max_depth`: `[10, 15, 20, 25]`
* For each of these values,
  * try different values of `n_estimators` from 10 till 200 (with step 10)
  * calculate the mean RMSE 
* Fix the random seed: `random_state=1`


What's the best `max_depth`, using the mean RMSE?

In [46]:
def train_rf2(df_train, y_train, n_estimators, depth):
    dicts = df_train.to_dict(orient='records')

    dv = DictVectorizer(sparse=True)
    X_train = dv.fit_transform(dicts)

    model = RandomForestRegressor(max_depth=depth, n_estimators=n_estimators, random_state=1, n_jobs=-1)
    model.fit(X_train, y_train)
    
    return dv, model


In [52]:
all_rmse = {}
for depth in [10, 15, 20, 25]:
    print('depth: %s' % depth)
    rmse_vals = []
    for i in range(10, 201, 10):
        dv, rf_model = train_rf2(df_train=df_train, y_train=y_train, n_estimators=i, depth=depth)
        y_pred = predict(df=df_val, dv=dv, model=rf_model)
        rmse_val = round(rmse(y_val, y_pred),3)
        print('%s -> %.3f' % (i, rmse_val))
        rmse_vals.append(rmse_val)
    all_rmse[depth] = np.mean(rmse_vals)

depth: 10
10 -> 0.251
20 -> 0.248
30 -> 0.246
40 -> 0.245
50 -> 0.246
60 -> 0.245
70 -> 0.245
80 -> 0.246
90 -> 0.246
100 -> 0.246
110 -> 0.245
120 -> 0.245
130 -> 0.245
140 -> 0.245
150 -> 0.245
160 -> 0.245
170 -> 0.245
180 -> 0.245
190 -> 0.245
200 -> 0.245
depth: 15
10 -> 0.246
20 -> 0.240
30 -> 0.238
40 -> 0.236
50 -> 0.236
60 -> 0.236
70 -> 0.236
80 -> 0.236
90 -> 0.235
100 -> 0.235
110 -> 0.235
120 -> 0.235
130 -> 0.235
140 -> 0.234
150 -> 0.234
160 -> 0.234
170 -> 0.234
180 -> 0.234
190 -> 0.235
200 -> 0.234
depth: 20
10 -> 0.244
20 -> 0.238
30 -> 0.236
40 -> 0.235
50 -> 0.235
60 -> 0.234
70 -> 0.234
80 -> 0.234
90 -> 0.234
100 -> 0.234
110 -> 0.234
120 -> 0.234
130 -> 0.234
140 -> 0.234
150 -> 0.234
160 -> 0.233
170 -> 0.233
180 -> 0.234
190 -> 0.234
200 -> 0.234
depth: 25
10 -> 0.245
20 -> 0.239
30 -> 0.237
40 -> 0.235
50 -> 0.235
60 -> 0.235
70 -> 0.235
80 -> 0.235
90 -> 0.235
100 -> 0.235
110 -> 0.234
120 -> 0.234
130 -> 0.234
140 -> 0.234
150 -> 0.234
160 -> 0.234
170 -> 0

In [53]:
all_rmse

{10: 0.24570000000000003,
 15: 0.2359,
 20: 0.23480000000000004,
 25: 0.23529999999999998}

# Question 5

We can extract feature importance information from tree-based models. 

At each step of the decision tree learning algorithm, it finds the best split. 
When doing it, we can calculate "gain" - the reduction in impurity before and after the split. 
This gain is quite useful in understanding what are the important features for tree-based models.

In Scikit-Learn, tree-based models contain this information in the
[`feature_importances_`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html#sklearn.ensemble.RandomForestRegressor.feature_importances_)
field. 

For this homework question, we'll find the most important feature:

* Train the model with these parameters:
  * `n_estimators=10`,
  * `max_depth=20`,
  * `random_state=1`,
  * `n_jobs=-1` (optional)
* Get the feature importance information from this model


What's the most important feature (among these 4)? 

* `total_rooms`
* `median_income`
* `total_bedrooms`
* `longitude`

In [54]:
dv, rf_model = train_rf2(df_train=df_train, y_train=y_train, n_estimators=20, depth=20)
importances = list(zip(dv.feature_names_, rf_model.feature_importances_))
df_importance = pd.DataFrame(importances, columns=['feature', 'gain'])
df_importance = df_importance.sort_values(by='gain', ascending=False)
df_importance

,feature,gain
4,median_income,0.336224
5,ocean_proximity=<1H OCEAN,0.218860
6,ocean_proximity=INLAND,0.146807
2,latitude,0.102264
3,longitude,0.085453
1,housing_median_age,0.031386
7,population,0.027086
9,total_rooms,0.021608
8,total_bedrooms,0.015231
0,households,0.015081


## Question 6

Now let's train an XGBoost model! For this question, we'll tune the `eta` parameter:

* Install XGBoost
* Create DMatrix for train and validation
* Create a watchlist
* Train a model with these parameters for 100 rounds:

```
xgb_params = {
    'eta': 0.3, 
    'max_depth': 6,
    'min_child_weight': 1,
    
    'objective': 'reg:squarederror',
    'nthread': 8,
    
    'seed': 1,
    'verbosity': 1,
}
```

Now change `eta` from `0.3` to `0.1`.

Which eta leads to the best RMSE score on the validation dataset?

* 0.3
* 0.1
* Both give equal value

In [68]:
import xgboost as xgb

In [67]:
dicts = df_train.to_dict(orient='records')
dv = DictVectorizer(sparse=True)
X_train = dv.fit_transform(dicts)
dtrain = xgb.DMatrix(X_train, label=y_train)

In [69]:
dicts = df_val.to_dict(orient='records')
dv = DictVectorizer(sparse=True)
X_val = dv.fit_transform(dicts)
dval = xgb.DMatrix(X_val, label=y_val)

In [73]:
xgb_params = {
    'eta': 0.3, 
    'max_depth': 6,
    'min_child_weight': 1,
    
    'objective': 'reg:squarederror',
    'nthread': 8,
    
    'seed': 1,
    'verbosity': 1,
}
model = xgb.train(xgb_params, dtrain, num_boost_round=100)
y_pred = model.predict(dval)
round(rmse(y_val, y_pred),3)


0.229

In [74]:
xgb_params = {
    'eta': 0.1, 
    'max_depth': 6,
    'min_child_weight': 1,
    
    'objective': 'reg:squarederror',
    'nthread': 8,
    
    'seed': 1,
    'verbosity': 1,
}
model = xgb.train(xgb_params, dtrain, num_boost_round=100)
y_pred = model.predict(dval)
round(rmse(y_val, y_pred),3)


0.232